**Fase 1: Conección y obtención de codigo fuente**

In [ ]:
# Hacer peticion HTTP
import requests
# Manipular código y guardar datos tabulares en archivo CSV
import pandas as pd

# url de la página web a «escrapear»
url = 'https://en.wikipedia.org/wiki/List_of_supermarket_chains'

# pasar "User-agent" para simular interacción con la página usando Navegador web
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

respuesta = requests.get(url, headers=headers)

# El código de respuesta <200> indicará que todo salió bien
print(respuesta)

<Response [200]>


In [ ]:
all_tables = pd.read_html(respuesta.content, encoding = 'utf8')

**Fase 2: Análisis de estructura HTML y extracción de datos**

In [ ]:
#Verificar el número de tablas que hay en la página web
print(f'Total de tablas encontradas: {len(all_tables)}')

Total de tablas encontradas: 3


In [ ]:
#Usar parámetro match para encontrar una tabla de intéres
matched_table = pd.read_html(respuesta.text, match='Ahold Delhaize')

# imprime numero de tablas que coinciden con parametro match
print(f'Total de tablas encontradas: {len(matched_table)}')

Total de tablas encontradas: 1


<ipython-input-30-b596e332959a>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matched_table = pd.read_html(respuesta.text, match='Ahold Delhaize')


In [ ]:
# Guardar tabla en variable con nombre semántico
multinational= matched_table[0]

# Verificamos si es la tabla que buscamos
multinational.tail(5)

,Company,Headquarters,Served countries (besides the headquarters),Map,Number of locations,Number of employees,Unnamed: 6
369,T&T Supermarket,Canada,NaN,NaN,33,NaN,NaN
370,Match,Belgium,"Luxembourg, France",NaN,217,NaN,NaN
371,C-market,Serbia,NaN,NaN,NaN,NaN,NaN
372,Tegut,Germany,NaN,NaN,275,7700,NaN
373,Comet,United Kingdom,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Eliminar las columnas "Map" y "Unnamed: 6"
multinational.drop(['Map', 'Unnamed: 6'], axis=1, inplace=True)

# Verificar si se eliminaron las columnas
multinational.tail(2)

,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
372,Tegut,Germany,NaN,275,7700
373,Comet,United Kingdom,NaN,NaN,NaN


In [ ]:
from unicodedata import normalize

In [ ]:
def remove_whitespace(x):
    """Funcion para normalizar datos con Unicode para luego quitar los espacios usando .replace().

    Argumentos de entrada: Nombre de columna o lista con nombres de columnas.
    Retorna: columna o columnas sin espacios en blanco
    """
    if isinstance(x, str):
        return normalize('NFKC', x).replace(' ', '')
    else:
        return x

In [ ]:
# Supongamos que numeric_cols es una lista de columnas numéricas en tu DataFrame
numeric_cols = ['Company','Headquarters','Number of locations', 'Number of employees', 'Served countries (besides the headquarters)']  # Reemplaza con tus nombres de columnas numéricas

# Define la función para eliminar espacios en blanco
def remove_whitespace(x):
    if isinstance(x, str):
        return x.strip()
    return x

# Aplicar map() a cada columna individual
for col in numeric_cols:
    multinational[col] = multinational[col].map(remove_whitespace)

# Imprimir cómo se ve la tabla después de la limpieza
print("Vista previa de la tabla después de eliminar espacios en blanco:")
print(multinational.head())  # Muestra las primeras 5 filas de la tabla

# Imprimir nombres de columnas para verificar si hay problemas de espacios
print("\nNombres de columnas después de eliminar espacios:")
print(multinational.columns.tolist())

# Imprimir tipos de datos para verificar que las columnas están correctamente formateadas
print("\nTipos de datos después de eliminar espacios:")
print(multinational.dtypes)

# Verificamos si se quitaron los espacios en blanco
multinational.head()



Vista previa de la tabla después de eliminar espacios en blanco:
          Company         Headquarters  \
0        7-Eleven  Japan United States   
1            Aeon                Japan   
2  Ahold Delhaize          Netherlands   
3       Aldi Nord              Germany   
4        Aldi Süd              Germany   

         Served countries (besides the headquarters) Number of locations  \
0  Australia, Canada, China, Cambodia, Denmark, H...               84500   
1  Australia, Cambodia, China, Hong Kong, India, ...               20008   
2  Belgium (as Albert Heijn and Delhaize), Czech ...                7659   
3  Belgium, Denmark, France, Luxembourg, Netherla...                5241   
4  Australia, Austria (as Hofer), China, Hungary,...                7178   

  Number of employees  
0              170000  
1            560,000+  
2              375000  
3               72811  
4              201361  

Nombres de columnas después de eliminar espacios:
['Company', 'Headquarters', 'S

,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
0,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",84500,170000
1,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008,"560,000+"
2,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659,375000
3,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",5241,72811
4,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",7178,201361


In [ ]:
# Mostrar tipo de datos de la tabla
multinational.dtypes

,0
Company,object
Headquarters,object
Served countries (besides the headquarters),object
Number of locations,object
Number of employees,object


In [ ]:
# Creamos diccionario y pasamos múltiples columnas con el tipo de dato a asignar
convert_dict = {
    'Company': 'string',
    'Headquarters': 'string',
    'Served countries (besides the headquarters)': 'string',
    'Number of locations': 'float64',
    'Number of employees': 'float64'
}

multinational = multinational.astype(convert_dict)

# Verificamos que las columnas con números tengan el tipo de dato numérico asignado
multinational.dtypes


,0
Company,string[python]
Headquarters,string[python]
Served countries (besides the headquarters),string[python]
Number of locations,float64
Number of employees,float64


In [ ]:
# Crear una columna 'row_id' con valores secuenciales
multinational['row_id'] = range(1, len(multinational) + 1)
# Luego, configurar 'row_id' como el índice
multinational.set_index('row_id', inplace=True)
# Verificar el cambio de índice
multinational.head(10)

,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
row_id,,,,,
1,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",84500.0,170000.0
2,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008.0,560000.0
3,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659.0,375000.0
4,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",5241.0,72811.0
5,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",7178.0,201361.0
6,Aldi,Germany (Süd and Nord),"Australia (Süd), Austria (Süd as Hofer), China...",12596.0,274172.0
7,Edeka,Germany,Denmark,13646.0,381000.0
8,Extra,Germany,<NA>,NaN,9700.0
9,Tengelmann Group,Germany,<NA>,4170.0,72714.0


**Fase 3: Guardado del Conjunto de Datos**

In [ ]:
# Guarda Dataframe a archivo CSV
multinational.to_csv('multinational.csv')

# Leamos el archivo para verificar su creacion
pd.read_csv('multinational.csv').head(3)

,row_id,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
0,1,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",84500.0,170000.0
1,2,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008.0,560000.0
2,3,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659.0,375000.0


In [ ]:
# Cargar o descargar archivos
from google.colab import files

# Descarga archivo con datos de tabla
files.download("multinational.csv")

print('Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...
